# Problem 0: **Cut the Rod to Maximize Profit**

Given the prices for rod pieces of every size between 1 inch and n inches, find the maximum total profit that can be made by cutting an n inches long rod inch into pieces.

### Example
---------------------------------------------------
```python
{ "pieces": [1,5,8,9] }
```

Output:
```python
10
```
```python
{ "pieces": [2,5,7,8,9] }
```
Output:
```python
12
```
### Notes
---------------------------------------------------
Constraints:
* 1 <= n <= 103
* 1 <= price of any sized piece of the rod <= 105

### Recursion
1. At each tree node, we need to choose all cut sizes that are possible within the given length. The length will be smaller on each tree level, so there will be 1 less choice per tree level.
2. We simply need to assess the max profit of
    1. The current max, OR
    2. The current size profit + the remaining size profit
3. To find the remaining size profit we'll recurse with the remaining size.
4. The Recurrence Relation appears to be
    - `T(n) = max(T(n-i) + profit of n), current_max)*i`
    - This is a decreasing function, so the Time Complexity will be len(pieces)^len(pieces). A tighter upper bound would be Order(N^n-1) (half the size of Order(N^n)). We're decreasing the set of choices by 1 every level in the tree hence n-1 power.

In [ ]:
def rod_cut_BF(prices, length=None):
    if length == None:
        length = len(prices)
    if length <= 0:
        return 0
    _max = 0
    for i in range(1, length+1):
        _max = max(_max, prices[i - 1] +
                   rod_cut_BF(prices, length-i))
    return _max

def rod_cut_MEMO(prices, length=None, memo={}): # Order(n^2) - we're looping inside recursion
    if length == None:
        length = len(prices)
    if length in memo:
        return memo.get(length)
    if length <= 0:
        return 0
    _max = 0
    for i in range(1, length+1):
        memo[length] = rod_cut_MEMO(prices, length-i)
        _max = max(_max, prices[i - 1] + memo.get(length))
    return _max

### Tabulation

* The first question to ask is which axis will show which variables?
    - The answer depends on the input, and the behavior of the problem. See my reference guide in dp_notes.
    - The essential rule here is, if the answer can be found based on a "Choices: Include/Exclude" paradigm then there is a deterministic answer.
    1. Rows will contain the choices we're making.
        * The number of pieces i'm asked to choose given length.
    2. Columns will contain the thing that's changing as we make choices.
        * The rod length: will change based on choices.
    3. Cells will contain the consequences of our choices given a length
        * The rod value at some length.
    *   | Values | Cut size | 0 | 1 | 2 | 3 | 4 | length |
        |--------|----------|---|---|---|---|---|--------|
        | 1      | 1        | 0 |   |   |   |   |        |
        | 5      | 2        | 0 |   |   |   |   |        |
        | 8      | 3        | 0 |   |   |   |   |        |
        | 9      | 4        | 0 |   |   |   |   |        |

        The _Values_ are obviously not necessary but i included them to be helpful.
    * Let's see how it's populated mid-way thru.
        | Values | Cut size | 0 | 1 | 2 | 3 | 4  | length |
        |--------|----------|---|---|---|---|----|--------|
        | 1      | 1        | 0 | 1 | 2 | 8 | 9  |        |
        | 5      | 2        | 0 | 1 | 5 | 8 | x  |        |
        | 8      | 3        | 0 |   |   |   |    |        |
        | 9      | 4        | 0 |   |   |   |    |        |

        Starting at `table[2][4]` **x**: The way to think here has a very formulaic process
    * Thinking process at cell `table[i][j]`: At length 4 (j), evaluate the choice to cut a size of 2 (i) off of given length-4 (j):
        * Option 1: **Include** the option. Calculate the inclusion in the following steps.
            1. Cutting a size of 2 means we immediately have a profit of `5`. So temporarily assign `table[i][j] = 5`.
            2. Next, we need to find the best possible value for the remaining size after cut off a size of 2.
            3. Stay in the current row (inclusive), and go back 2 columns (j - 2). This step is simulating a cut, and then looking at the remainder (4-2 = length 2). The cell value at j = 2 is the best answer we have for **any cuts of any size** <= j (2).
                - Where did that best answer at the remainder cell `table[i][2]` come from? It came from the bottom-up work we've already done up to that point; Up to this point we've finished analyzing all cuts of size 1 & 2 (except the one we're doing now) and we saved all the best answers we found so far all of the cell's with a value.

        * Option 2: **Exclude** the option to cut into 3 pieces.
            - Assign as: `table[i][j] = table[i-1][j]`

In [43]:
def rod_cut_DP(prices):
    n = len(prices)
    t = [[0]*(n+1) for _ in range(n)]
    for j in range(1, n+1):
        t[0][j] = prices[j-1]
    for i in range(1, n):
        for j in range(n+1):
            if j == 0:
                t[i][j] = 0
            elif i <= j:
                t[i][j] = max(
                    t[i][j-i] + prices[i-1], # include cut at starting point i
                    t[i-1][j]                # exclude cut at ...
                )
            else:
                t[i][j] = t[i-1][j]         # j length is not long enough to cut at starting point i
    return t[-1][-1]
rod_cut_DP([1, 5, 6, 8, 9])

11

# IK Problems
---------------------------------------------------

## Problem 1: **Pascals Triangle**

Pascal’s triangle is a triangular array of the binomial coefficients. Write a function that takes an integer value n as input and returns a two-dimensional array representing pascal’s triangle.

* `pascalTriangleArray` is a two-dimensional array of size n * n, where
* `pascalTriangleArray[i][j] = BinomialCoefficient(i, j); if j <= i`,
* `pascalTriangleArray[i][j] = 0; if j > i`

### Example
---------------------------------------------------
```python
{ "n": 4 }
```

Output:
```python
[
[1],
[1, 1],
[1, 2, 1],
[1, 3, 3, 1]
]
```
### Notes
---------------------------------------------------
* All values in the 2D output array `result` must be modulo with (10**9 + 7) and size of `result[i]` for `0 <= i < n` should be `(i + 1)` i.e. 0s for `pascalTriangleArray[i][j] = 0`; if `j > i`, should be ignored.

Constraints:
* 1 <= `n` <= 1700

---------------------------------------------------
### Notes
1. How can we break the problem into subproblems recursively?
    1. What choices are we asked to make?
        - Yes or No
    1. We want to go as deep as `n`. So what if we start with 1.

In [ ]:
def find_pascal_triangle_recursive(n):
    pass

In [44]:
"""
  | 0 | 1 | 2 | 3 | j
  |---|---|---|---|
0 | 1 | 1 | 1 | 1 |
1 | 1 | 2 | 3 |   |
2 | 1 | 3 |   |   |
3 | 1 |   |   |   |
i

[
[1],
[1, 1],
[1, 2, 1],
[1, 3, 3, 1]
]

[
[1],
[1],
[1],
[1]
]
"""

def find_pascal_triangle(n):
    mod = 10**9 + 7
    t = [[] for _ in range(n)]
    for i in range(0, n):
        for j in range(i+1):  # 0, 1 | 0, 1, 2 | 0, 1, 2, 3
            if j > 0 and j < len(t[i-1]):  # if 1 > 0 and 1 < 1:
                t[i].append((t[i-1][j] + t[i-1][j-1]) % mod)
            else:
                t[i].append(1)
    return t


find_pascal_triangle(10)

[[1],
 [1, 1],
 [1, 2, 1],
 [1, 3, 3, 1],
 [1, 4, 6, 4, 1],
 [1, 5, 10, 10, 5, 1],
 [1, 6, 15, 20, 15, 6, 1],
 [1, 7, 21, 35, 35, 21, 7, 1],
 [1, 8, 28, 56, 70, 56, 28, 8, 1],
 [1, 9, 36, 84, 126, 126, 84, 36, 9, 1]]

---------------------------------------------------

## Problem 2: **Minimum Sum Path In A Triangle**
Find the minimum path sum from top to bottom in a given triangular structure of numbers. For each step, movement to the adjacent cells down from the current position is allowed i.e. from any index `i `in the current row, movement to index `i` or `i + 1` in the next row is a valid movement.

### Example
---------------------------------------------------
```python
{
    "triangle": [
        [2],
        [-4, -3],
        [8, 3, 9],
        [4, 2, 1, 6]
    ]
}
```

Output:
```python
2
```

The Triangle Looks like
```python

             2
        -4      -3
    8       3       9
4       2       1       6
```

The minimum path sum from top to bottom is `2 + -4 + 3 + 1 = 2`.


### Notes
---------------------------------------------------
The `i`th row in the input list will have `i` + 1 elements.

Constraints:
* 1 <= size of the input list <= 103
* -103 <= any element of the input list <= 103

In [52]:
def get_minimum_path_sum(triangle):
    n = len(triangle)
    for i in range(n-2, -1, -1):
        for j in range(i+1):
            s = triangle[i][j] + min(triangle[i+1][j],  triangle[i+1][j+1])
            triangle[i][j] = s
    return triangle[0][0]


get_minimum_path_sum([
    [50],
    [30, 30],
    [-5, -5, -5],
    [32, 32, 32, 32],
    [-10, -10, -10, -10, -10]
])


97

---------------------------------------------------

## Problem 3: **Minimum Coins**
Given a variety of coin types defining a currency system, find the minimum number of coins required to express a given amount of money. Assume infinite supply of coins of every type.


### Example
---------------------------------------------------
```python
{
    "coins": [1, 3, 5],
    "value": 9
}
```

Output:
```python
3
```
Here are all the unique ways to express 9 as a sum of coins 1, 3 and 5:

* 1, 1, 1, 1, 1, 1, 1, 1, 1
* 1, 1, 1, 1, 1, 1, 3
* 1, 1, 1, 1, 5
* 1, 1, 1, 3, 3
* 1, 3, 5
* 3, 3, 3
Last two ways use the minimal number of coins, 3.


### Notes
---------------------------------------------------
There will be no duplicate coin types in the input.

Constraints:
* 1 <= number of coin types <= 102
* 1 <= coin value <= 102
* 1 <= amount of money to express <= 104



In [9]:
"""
 Time Complexity (Recursion): v**c
                    9
                8       9
        7   8
    6   7
    For every node, we have to make len(coins) decisions

    Recurrence Relation
              0                         v == 0
    T(c, v) = min(T(c, v-c[i] + 1))     |c|,
                                        0 <= i < |c|
                        0   1   2   3   4   5   6   7   8   9   Value
    Coins (i)   0   1   0   1   2   3   4   5   6   7   8   9
                1   3   0   1   2   1   2   3   2   3   4   3
                2   5   0   1   2   1   2   1   2   3   2   3
    T(C*v)
"""
def minimum_coins(coins, value):
    n = value + 1
    t = [0 for _ in range(n)]
    for j in range(n):
        for i in range(len(coins)):
            """
            Coins need to be handled on every j because we need to visit every
            Neighbor of the current DAG node before proceeding further given
            that the problem is asking for the Minimum
            """
            coin = coins[i]
            if j == 0 or coin > j:
                continue
            else:
                exclude_coin = t[j]
                include_coin = t[j-coin] + 1
                if not exclude_coin:
                    t[j] = include_coin
                else:
                    t[j] = min(t[j], include_coin)
    return t[-1]

args = {
    "coins": [22, 14, 1, 18],
    "value": 889
}

args_2 = {
    "coins": [1, 3, 5],
    "value": 9
}

minimum_coins(*args.values())

43

---------------------------------------------------

## Problem 4: **Robbery**
There are `n` houses built in a line, each of which contains some value in it. A thief is going to steal the maximal value in these houses, but he cannot steal in two adjacent houses because the owner of a stolen house will tell his two neighbours on the left and right side. What is the maximal stolen value?

For example, if there are four houses with values `[6, 1, 2, 7]`, the maximal stolen value is 13, when the first and fourth houses are stolen.


### Example
---------------------------------------------------
```python
{
"values": [6, 1, 2, 7]
}
```

Output:
```python
13
```
Steal from the first and the last house.


### Notes
---------------------------------------------------
There will be no duplicate coin types in the input.

Constraints:
* 1 <= `n` <= 105
* 1 <= `values[i]` <= 104, for all `i`.

In [85]:
def maximum_stolen_value_RECURSIVE(v, cix=0, s=None):
    if s is None:
        s = set([i for i in range(len(v))])
        s.remove(cix)
    elif not s:
        return v[cix]
    _max = v[cix]
    s_copy = s.copy()
    for i in s:
        if abs(cix - i) == 1:
            continue
        s_copy.remove(i)
        r_max = maximum_stolen_value_RECURSIVE(v, i, s_copy)
        _max = max(_max, r_max, v[cix] + v[i])
        s_copy.add(i)
    return _max

def maximum_stolen_value_DP(values):
    n = len(values) + 1
    t = [[0]*n for _ in range(n)]
    for j in range(1, n):
        diag_len = n - j
        for i in range(diag_len):
            p = i+j
            if j == 1:
                t[i][p] = values[i]
            else:
                for d in range(j):
                    t[i][p] = max(t[i][p], t[i][d] + t[i+1+d][p])
    return t[0][-1]

# maximum_stolen_value_DP([6, 1, 2, 7]) # 13
# maximum_stolen_value_DP([7, 9, 9, 6, 4, 3, 8, 9, 5, 3, 3, 5]) # 38
maximum_stolen_value_DP([8, 7, 10, 7, 8, 9, 4, 5, 7, 2, 7, 7, 9, 8, 4, 7, 2])  # 60

60

### NOTE
1. Both of the above solutions work but the recursive solution is rather slow.
2. The DP solution also works but time's out for very large inputs. Also, the Matrix seems to consume quite a lot of memory.
3. The recursive solution below is memoized-DP and is the faster of the 3.

### Recursive Thinking
1. For Recursive-Memoized-DP (REMO DP) the idea is to think that you are a _time traveller_ trying to travel to a distant moment in the past when the inputs are exactly equal to 0. If the moment in time has an input value < 0 then you've gone too far and you need to stop looking. If the input is > 0 then you still need to keep looking further into the past.
2. Now we create an _expression_ to define the terms from the current present moment, and try to imagine what must the expression say to travel back in time? Well in most DP questions, we're trying to understand the consequences of our choices. And choices can be most aptly compared to a _decision tree_. This decision tree has branches, and each branch represents a move further into the past by making that branch's choice.
3. For this problem, we're wanting to travel back in time to a point where we first started to choose to rob a house. So we imagine a moment when we were looking down a neighbood street seeing a long street with lots of houses, and we decided to start robbing. Our choices at that point are to either choose the first house or not. If we choose the first house `index-2 ... + values[index]`, we must add the value of that robbery to our counter.  Then we need to move to the house two houses away, skipping the neighbor.  But what if we chose NOT to rob the first house? Well in that case, we only skip that one house, and we go to the neighbor of the house we just skipped! `index - 1` and re-evaluate the situation.
4. That's it!! That's our entire requirement for travelling back in time and seeing the whole tree of possibilities.

**The key** is to formulate the recursive calls from the _present_ looking into the _past_ and understanding the conditions at `t=0` moving toward `t=N`.

In [3]:
def  maximum_stolen_value(values, index=None, memo=None):
    if len(values) == 1: return values[0]
    if index < 0: return 0
    if index is None:
        index = len(values) - 1
        memo = {0: values[0], 1: max(values[0], values[1])}
    if index in memo: return memo.get(index)
    _max = max(
        maximum_stolen_value(values, index-1, memo), # dont rob
        maximum_stolen_value(values, index-2, memo) + values[index]) # rob
    memo[index] = _max
    return memo[index]

maximum_stolen_value([8, 7, 10, 7, 8, 9, 4, 5, 7, 2, 7, 7, 9, 8, 4, 7, 2])


60

---------------------------------------------------

## Problem 5: **Color The Fence**
There is a fence with `n` posts and there are `k` different colors. Find the number of ways the fence can be painted, where:

* Every post must be painted with exactly one color.
* Three or more consecutive posts are not painted with the same color.
The answer can be very large. So, return answer modulo 109 + 7.

### Example 1
---------------------------------------------------
```python
{
    "n": 3,
    "k": 2
}
```

Output:
```python
6
```

These are the possible ways of coloring the fence.
```python
{
    Color1-Color1-Color2,
    Color1-Color2-Color1,
    Color1-Color2-Color2,
    Color2-Color1-Color1,
    Color2-Color1-Color2,
    Color2-Color2-Color1
}
```

### Example 2
---------------------------------------------------
```python
{
    "n": 3,
    "k": 7
}
```

Output:
```python
336
```

### Notes
---------------------------------------------------
There will be no duplicate coin types in the input.

Constraints:
* 1 <= n <= 105
* 1 <= k <= 109

### Solution Thinking
1. Just as in the last problem we're going to think recursively from the _present_ moment into the _past_ and try to imagine: _What are the set of choices that I can make at `t=0` to `t=n`?_
2. It seems there are 2 sets of choices.
    1. We can color the first two posts the same color: This would fall into the first constraint - _only two posts can be painted with the same color_
    2. We can NOT color the first two posts the same color: This would fall into the second constraint - _every post must be painted with one color_.
    3. **NOTE** It should be noticed that we don't need the problem to explicitly define the choices. Rather we can extract an implied set of choices given a single constraint. Choice 1 is the choice that falls into that constraint. Choice 2 is the choice that allows us to _avoid_ that constraint.
3. Now that we have our choices, we must figure out how to express the `state` of remaining choices. In the **Rober** question we defined the expression of the change in state as `index - 1` or `index - 2 + robbed_value`.  In that case, we were saying that the relationship between the two choices was a simple removal of that choice from the choice pool.
4. It's also important to note that some moments in time have no relation to previous choices made as in the case of the robber problem. The history of choices the robber made, were not needed to be remembered by the robber to make a choice in the present moment. _This is not always the case_. This problem does require the time traveller to remember his previous choices, but only to a certain extent. For this problem let's refer to our time traveller as "the painter". The piainter is time travelling on a timeline that we'll describe as `t=n` to `t=0`. In ther robber problem, when the robber was at `t=n-1`, he didn't need to remember the choice he made previously at `t=n`. However, in this problem it's different.
5. The first thing to remember is how the choice pool changes when the painter wants to paint two posts the same color. The problem states that the painter cannot choose more than two colors in a row. This means, if we're at `t=n-1` we need to remember the choice made at `t=n` so that we know what choices are available at `t=n-1`. Also, if the painter does choose two posts of the same color, then chooses a new color of the 2 remaining choices available, his available choices of color reset at `t=n-3`! He can paint the fourth post any of the three colors available since the last two posts are different colors. This means that `t=n-2` needs to know about `t=n-1` choice. Ok, now moving forward one more step, at `t=n-2` into `t=n-3` the choices are exactly the same when the painter was at `t=n`. Now that the painter has found a repetition of choice options across three time units (`t=n`, `t=n-1` and `t=n-2`) he knows how many previous choices he has to remember: 3. The painter is now ready to summarize that pattern into a formula in order to properly travel back in time.
6. Let's summarize this first part with two posts the same color - starting out our choices are: `k=3` and `n=5`
    - When the painter is at `t=n` he can make `k` choices (three color choices).
    - When the painter is at `t=n-1` he can make `k-1` choices (only two colors).
    - When the painter is at `t=n-2` his choice of color resets and he has `k` choices again.
7. The second and more easy part is if the painter chooses a different color each post. In this case, then at `t=n` the painter has `k` choices. At `t=n-1` he has `k-1` choices since he cannot paint the current post the same color as the previous post choice at `t=n`. Again, at `t=n-2` his choices of color is `k-1` since he still can't paint the current post the same color as before.
8. Let's summarize this second part with two posts different colors - start out our choices are: `k=3` and `n=5`
    - When the painter is at `t=n` he can make `k` choices (three color choices).
    - When the painter is at `t=n-1` he can make `k-1` choices (only two colors).
    - When the painter is at `t=n-2` he can make `k-1` choices (only two colors).
9. Now we have to combine the first and second part together to express the formula of all choices for both types of scenarios.
    1. When the painter is at `t=n` he can:
        * Choose `k` colors if he's planning to paint two posts the SAME color.
        * Choose `k` colors if he's planning to paint every post a DIFFERENT color.
    2. When the painter is at `t=n-1` he can:
        * Choose `k` colors if he's planning to paint two posts the SAME color
        * Choose `k-1` colors if he's planning to paint every post a DIFFERENT color.
    3. When the painter is at `t=n-2` he can:
        * Choose `k-1` colors if he's planning to paint two posts the SAME color.
        * Choose `k-1` colors if he's planning to paint every post a DIFFERENT color.
    4. When the painter is at `t=n-3` he can:
        * Choose `k` colors if he's planning to paint two posts the SAME color.
        * Choose `k-1` colors if he's planning to paint every post a DIFFERENT color.
        * We inclue this 4'th time step `t=n-3` as clear way of showing the that the pattern is now repeating the same flow from step #2. Because of this repeat, we don't need to express this step 4 in our forumla.
10. The Formula
    * time unit: (same + different) * (choices to remember)
    * `t=n`: `(0 + k)` = `(3 + 3)` = `6` -> nothing to remember
    * `t=n-1`: `(k + k-1)` = `(3 + 2) * 6` = `30`
    * `t=n-2`: `(k-1 + k-1)`
    * Final Formula = `(2 + k-1) * (k + k-1)  * (k + k)`
    * Final Answer when `k=3`: `(3 + 3) * (3 + 2) * (3 + 3)` = `6 * 5 * 6` = `180`

In [ ]:
def number_of_ways(n, k):
    """
    Args:
     n(int32)
     k(int32)
    Returns:
     int32
    """
    # Write your code here.
    return 0


---------------------------------------------------

## Problem 6: **Knights Tour on a Phone Keypad**
Given a phone keypad as shown below:

1 2 3
4 5 6
7 8 9
– 0 –

How many different phone numbers of given length can be formed starting from the given digit? The constraint is that the movement from one digit to the next is similar to the movement of the Knight in chess.

For example, if we are at 1, then the next digit can be either 6 or 8; if we are at 6 then the next digit can be 1, 7 or 0.

Repetition of digits is allowed, e.g. 1616161616 is a valid number.
There is no need to list all possible numbers, just find how many they are.
Find a polynomial-time solution, based on Dynamic Programming.

### Example 1
---------------------------------------------------
```python
{
    "startdigit": 1,
    "phonenumberlength": 2
}
```

Output:
```python
2
```

Two possible numbers of length 2: 16, 18.


### Example 2
---------------------------------------------------
```python
{
    "startdigit": 1,
    "phonenumberlength": 3
}
```

Output:
```python
5
```
The possible numbers of length 3: 160, 161, 167, 181, 183


### Notes
---------------------------------------------------
* There are two input parameters: `startdigit` and `phonenumberlength`, denoting the starting digit and the required length respectively.
* Output is a long integer denoting the total number of valid phone numbers that can be formed.

Constraints:
* 0 <= `startdigit` <= 9
* 1 <= `phonenumberlength` <= 30

In [ ]:

def count_phone_numbers_of_given_length(startdigit, phonenumberlength):
    """
    Args:
     startdigit(int32)
     phonenumberlength(int32)
    Returns:
     int64
    """
    # Write your code here.
    return 0


---------------------------------------------------

## Problem 7: **Levenshtein Distance**
Given two words `word1` and `word2`, find the minimum number of steps required to convert `word1` to `word2`. (each operation is counted as 1 step.)

You have the following 3 operations permitted on a word:
* Insert a character
* Delete a character
* Replace a character

The minimum number of steps required to convert `word1` to `word2` with the given set of allowed operations is called edit distance. e.g. Minimum edit distance between the words 'kitten' and 'sitting', is 3.

* kitten → sitten (substitution of "s" for "k")
* sitten → sittin (substitution of "i" for "e")
* sittin → sitting (insertion of "g" at the end)
Read more about edit distance [here](https://en.wikipedia.org/wiki/Edit_distance).

### Example 1
---------------------------------------------------
```python
{
    "word1": "cat",
    "word2": "bat"
}

```

Output:
```python
1
```

Two possible numbers of length 2: 16, 18.


### Example 2
---------------------------------------------------
```python
{
    "word1": "qwe",
    "word2": "q"
}
```

Output:
```python
2
```

### Notes
---------------------------------------------------
Constraints:
* 1 <= length of the strings word1 and word2 <= 105
* word1 and word2 contains lower case alphabets from 'a' to 'z'.

In [ ]:

def levenshtein_distance(word1, word2):
    """
    Args:
     word1(str)
     word2(str)
    Returns:
     int32
    """
    # Write your code here.
    return 0


---------------------------------------------------

## Problem 8: **Word Break Count**
Given a dictionary of words and a string `txt`, find the number of ways the string can be broken down into the dictionary words. Return the answer modulo 10^9 + 7.

### Example 1
---------------------------------------------------
```python
{
    "dictionary": ["kick", "start", "kickstart", "is", "awe", "some", "awesome"],
    "txt": "kickstartisawesome"
}

```

Output:
```python
4
```

Here are all four ways to break down the string into the dictionary words:

1. kick start is awe some
2. kick start is awesome
3. kickstart is awe some
4. kickstart is awesome
4 % 1000000007 = 4 so the correct output is 4.


### Notes
---------------------------------------------------
Constraints:
* 1 <= number of words in the dictionary <= 2 * 105
* 1 <= length of any dictionary word <= 102
* 1 <= length of the string txt <= 2 * 103
* Dictionary words and the string txt all consist of lowercase latin characters only (no whitespace, in particular).

In [ ]:

def word_break_count(dictionary, txt):
    """
    Args:
     dictionary(list_str)
     txt(str)
    Returns:
     int32
    """
    # Write your code here.
    return 0


---------------------------------------------------

## Problem 9: **Equal Subset Partition**
Given an array `s` of `n` integers, partition it into two non-empty subsets, `s1` and `s2`, such that the sum of all elements in `s1` is equal to the sum of all elements in `s2`. Return a boolean array of size `n` where `i`-th element is 1 if `i`-th element of s belongs to `s1` and 0 if it belongs to `s2`.


### Example 1
---------------------------------------------------
```python
{
    "s": [10, -3, 7, 2, 1, 3]
}

```

Output:
```python
[1, 1, 0, 0, 0, 1]
```
There are multiple partitionings where s1 sums up to 10 and s2 sums up to 10; they are all correct answers:

`s1 = [ 10 , -3 , 3 ]` and `s2 = [ 7 , 2 , 1 ]` (Sample output)

`s1 = [ 7 , 2 , 1 ]` and `s2 = [ 10 , -3 , 3 ]`

`s1 = [10]` and `s2 = [-3, 3, 7, 2, 1]`

`s1 = [-3, 3, 7, 2, 1]` and `s2 = [10]`

`s1 = [10, -3, 2, 1]` and `s2 = [7, 3]`

`s1 = [7, 3]` and `s2 = [10, -3, 2, 1]`


### Notes
---------------------------------------------------
* Any valid answer will be accepted.
* If such partitioning is not possible, return an empty array.

Constraints:
* 1 <= `n` <= 100
* -100 <= elements in `s` <= 100

In [ ]:

def equal_subset_sum_partition(s):
    """
    Args:
     s(list_int32)
    Returns:
     list_bool
    """
    # Write your code here.
    return []


---------------------------------------------------

## Problem 10: **Longest Common Subsequence**
Find the longest common subsequence of two strings.

A subsequence is a sequence that can be derived from another sequence by deleting zero or more elements without changing the order of the remaining elements.


### Example 1
---------------------------------------------------
```python
{
    "a": "ABCDE",
    "b": "AECBD"
}

```

Output:
```python
"ABD"
```
Subsequence "ABD" can be derived from the first string by deleting characters "C" and "E". From the second string it can be derived by deleting "E" and "C". No common subsequence longer than three characters exists in the two given strings. "ACD" is another common subsequence of length three; it is also a correct answer.


### Notes
---------------------------------------------------
* If a nonempty common subsequence cannot be found, return "-1".
Constraints:
* 1 <= length of each of the input strings <= 400
* Input strings consist of the alphanumeric characters

In [ ]:
def lcs(a, b):
    """
    Args:
     a(str)
     b(str)
    Returns:
     str
    """
    # Write your code here.
    return ''


---------------------------------------------------

## Problem 11: **Cut The Rod To Maximize Profit**
Given the prices for rod pieces of every size between 1 inch and `n` inches, find the maximum total profit that can be made by cutting an `n` inches long rod inch into pieces.


### Example 1
---------------------------------------------------
```python
{
"price ": [1, 5, 8, 9]
}
```

Output:
```python
10
```

The rod can be cut in the ways given below:

* `1 + 1 + 1 + 1` inches will cost `price[0] + price[0] + price[0] + price[0] = 4`
* `1 + 1 + 2` inches will cost `price[0] + price[0] + price[1] = 7`
* `1 + 3` inches will cost `price[0] + price[2] = 9`
* `2 + 2` inches will cost `price[1] + price[1] = 10`
* One piece of 4 inches will cost` price[3] = 9`
The maximum profit is obtained by cutting it into two pieces 2 inches each.

### Notes
---------------------------------------------------
Constraints:
* 1 <= `n` <= 103
* 1 <= price of any sized piece of the rod <= 105

```markdown
       j
| 0  | 1  | 2 | 3 | 4 |
|----|----|---|---|---|
|    |    |   |   |   |
  i
```


In [7]:
from math import inf
def get_maximum_profit(price):
    """
    Args:
     price(list_int32)
    Returns:
     int32
    """
    # Write your code here.
    return 0

def cut_the_rope(n):
    table = [0 for i in range(n+1)]
    for i in range(2, n+1):  # n + 1 because we want to SEE length n
        _max = 0
        for j in range(1, i): # By Iterating into i, we ensure j will always be less than i. This technique is ubiquitous in tab.DP
            remainder = i-j  # reamining size after cut of size j
            best_cut_prev_size = table[i-j] # best value from prev. size j cut.
            curr_max_len = j * max(remainder, best_cut_prev_size) # we multiply by j because...
            _max = max(_max, curr_max_len)
        table[i] = _max
    return table[-1]

cut_the_rope(4)


4

---------------------------------------------------

## Problem 12: **Number Of Ways To Make Change**
Given a variety of coin denominations existing in a currency system, find the total number of ways a given amount of money can be expressed using coins in that currency system.

Assume infinite supply of coins of every denomination. Return answer modulo 1000000007.

### Example 1
---------------------------------------------------
```python
{
"coins ": [1, 2, 3],
"amount": 3
}
```

Output:
```python
3
```

The three ways are:

* Use the coin with denomination 1 three times.
* Use the coin with denomination 3 once.
* Use the coin with denomination 2 once and coin with denomination 1 once.

### Notes
---------------------------------------------------
* Two ways are considered different if they use a different number of coins of any particular denomination.

Constraints:
* 1 <= total number of denominations <= 102
* 1 <= denomination of a coin <= 104
* 1 <= amount to be expressed <= 104

In [ ]:

def number_of_ways(coins, amount):
    """
    Args:
     coins(list_int32)
     amount(int32)
    Returns:
     int32
    """
    # Write your code here.
    return 0